In [3]:
!git clone https://github.com/ignacio-ave/ADA-Portafolio.git


Cloning into 'ADA-Portafolio'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 67 (delta 0), reused 7 (delta 0), pack-reused 60
Receiving objects: 100% (67/67), 82.94 MiB | 33.08 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [4]:
# Importamos las bibliotecas necesarias
import numpy as np
import struct

# Funciones para cargar los datos MNIST
def load_images(filename):
    with open(filename, "rb") as f:
        data = f.read()
        _, num_images, rows, cols = struct.unpack(">IIII", data[:16])
        images = np.frombuffer(data[16:], dtype=np.uint8).reshape(num_images, rows * cols)
        images = images.astype(np.float32) / 255.0  # Normalización
    return images

def load_labels(filename):
    with open(filename, "rb") as f:
        data = f.read()
        _, num_labels = struct.unpack(">II", data[:8])
        labels = np.frombuffer(data[8:], dtype=np.uint8)
    return labels

# Cargamos los datos de entrenamiento y de prueba
path = input("Ingrese la ruta de la carpeta MNIST DATABASE: ")
X_train = load_images(path+"/train-images.idx3-ubyte")
y_train = load_labels(path+"/train-labels.idx1-ubyte")
X_test = load_images(path+"/t10k-images.idx3-ubyte")
y_test = load_labels(path+"/t10k-labels.idx1-ubyte")

# Convertimos las etiquetas a representación one-hot
num_classes = 10
y_train_one_hot = np.eye(num_classes)[y_train]
y_test_one_hot = np.e
ye(num_classes)[y_test]

# Verificamos las dimensiones de los datos
X_train.shape, y_train_one_hot.shape, X_test.shape, y_test_one_hot.shape


FileNotFoundError: [Errno 2] No such file or directory: 'ADA-Portafolio/Backpropagation/MNIST DATABASE/train-images.idx3-ubyte'